# PostgreSQL Database Connection & Loading Data to Database

In [2]:
import pandas as pd
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass

In [36]:
# Read in CSV with model sentiment predictions
tweet_df = pd.read_csv('../res/big_data_prediction_ml_model.csv')

In [37]:
tweet_df.head()

,Unnamed: 0,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,...,fips_state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink,sentiment
0,0,1588320083335467009,2022-11-03 23:59:52,"""@UnnecRoughness A is Levis. \nB is KJ Jeffers...",Reply Tweet,NaN,['@UnnecRoughness'],531999121,NaN,NaN,...,5.0,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...,neutral
1,1,1588320019259469824,2022-11-03 23:59:36,"""Republican candidate's kids are almost KILLED...",Original Tweet,NaN,NaN,1586027897579802624,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...,neutral
2,2,1588320006840160256,2022-11-03 23:59:33,"""@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy...",Reply Tweet,NaN,"['@Jupiter62214807', '@cjstheman_611', '@BMC_M...",783106891274596352,NaN,NaN,...,54.0,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...,neutral
3,3,1588319891446272001,2022-11-03 23:59:06,"""@davidhogg111 It’s 2022, get new talking poin...",Reply Tweet,NaN,['@davidhogg111'],1511111650199412739,NaN,NaN,...,26.0,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...,neutral
4,4,1588319866385477632,2022-11-03 23:59:00,"""Watch: Gunman opens fire on Imran Khan https:...",Original Tweet,NaN,NaN,287297482,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...,neutral


In [38]:
# Drop unlabeled index column
tweet_df.drop('Unnamed: 0', axis=1, inplace=True)

In [74]:
# Check non-null counts and dtypes
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72331 entries, 0 to 72330
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tweet_id        72331 non-null  object 
 1   date_created    72331 non-null  object 
 2   full_text       72331 non-null  object 
 3   tweet_type      72331 non-null  object 
 4   hashtags        10552 non-null  object 
 5   mentions        54574 non-null  object 
 6   user_id         72331 non-null  object 
 7   city            39278 non-null  object 
 8   county          37758 non-null  object 
 9   fips_county     37758 non-null  object 
 10  state           62966 non-null  object 
 11  fips_state      62966 non-null  object 
 12  country         72331 non-null  object 
 13  latitude        72331 non-null  float64
 14  longitude       72331 non-null  float64
 15  reply_count     72331 non-null  int64  
 16  quote_count     72331 non-null  int64  
 17  likes_count     72331 non-null 

In [45]:
# Convert tweet_id and user_id columns to string from integer
tweet_df['tweet_id'] = tweet_df['tweet_id'].astype(str)
tweet_df['user_id'] = tweet_df['user_id'].astype(str)

In [46]:
# Confirm dtype changes
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72331 entries, 0 to 72330
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tweet_id        72331 non-null  object 
 1   date_created    72331 non-null  object 
 2   full_text       72331 non-null  object 
 3   tweet_type      72331 non-null  object 
 4   hashtags        10552 non-null  object 
 5   mentions        54574 non-null  object 
 6   user_id         72331 non-null  object 
 7   city            39278 non-null  object 
 8   county          37758 non-null  object 
 9   fips_county     37758 non-null  float64
 10  state           62966 non-null  object 
 11  fips_state      62966 non-null  float64
 12  country         72331 non-null  object 
 13  latitude        72331 non-null  float64
 14  longitude       72331 non-null  float64
 15  reply_count     72331 non-null  int64  
 16  quote_count     72331 non-null  int64  
 17  likes_count     72331 non-null 

In [47]:
# Confirm changes
tweet_df.head()

,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,fips_county,...,fips_state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink,sentiment
0,1588320083335467009,2022-11-03 23:59:52,"""@UnnecRoughness A is Levis. \nB is KJ Jeffers...",Reply Tweet,NaN,['@UnnecRoughness'],531999121,NaN,NaN,NaN,...,5.0,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...,neutral
1,1588320019259469824,2022-11-03 23:59:36,"""Republican candidate's kids are almost KILLED...",Original Tweet,NaN,NaN,1586027897579802624,NaN,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...,neutral
2,1588320006840160256,2022-11-03 23:59:33,"""@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy...",Reply Tweet,NaN,"['@Jupiter62214807', '@cjstheman_611', '@BMC_M...",783106891274596352,NaN,NaN,NaN,...,54.0,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...,neutral
3,1588319891446272001,2022-11-03 23:59:06,"""@davidhogg111 It’s 2022, get new talking poin...",Reply Tweet,NaN,['@davidhogg111'],1511111650199412739,NaN,NaN,NaN,...,26.0,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...,neutral
4,1588319866385477632,2022-11-03 23:59:00,"""Watch: Gunman opens fire on Imran Khan https:...",Original Tweet,NaN,NaN,287297482,NaN,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...,neutral


### Correcting FIPS Codes in CSV with Model Predictions

In [23]:
# Read in CSV without model predictions as string to get correct fips codes columns (converted to float in predictions df)
fips_df = pd.read_csv('../res/big_data_tweets.csv', dtype='str')

In [25]:
# Select only tweet_id and fips code columns to confirm correct values
fips_df = fips_df.loc[:, ['tweet_id', 'fips_county', 'fips_state']]

fips_df.head(10)

,tweet_id,fips_county,fips_state
0,1588320083335467009,NaN,05
1,1588320019259469824,NaN,NaN
2,1588320006840160256,NaN,54
3,1588319891446272001,NaN,26
4,1588319866385477632,NaN,NaN
5,1588319827651104769,09003,09
6,1588319810181529602,36047,36
7,1588319807375433728,06045,06
8,1588319706343124992,06053,06
9,1588319697808019456,NaN,NaN


In [35]:
# Check counts and dtypes
fips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72331 entries, 0 to 72330
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     72331 non-null  object
 1   fips_county  37758 non-null  object
 2   fips_state   62966 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [48]:
# Assign corrects fips code value columns to fips code columns in the predicition dataframe.
tweet_df['fips_county'] = fips_df['fips_county']
tweet_df['fips_state'] = fips_df['fips_state']

In [49]:
# Confirm counts and dtypes are correct
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72331 entries, 0 to 72330
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tweet_id        72331 non-null  object 
 1   date_created    72331 non-null  object 
 2   full_text       72331 non-null  object 
 3   tweet_type      72331 non-null  object 
 4   hashtags        10552 non-null  object 
 5   mentions        54574 non-null  object 
 6   user_id         72331 non-null  object 
 7   city            39278 non-null  object 
 8   county          37758 non-null  object 
 9   fips_county     37758 non-null  object 
 10  state           62966 non-null  object 
 11  fips_state      62966 non-null  object 
 12  country         72331 non-null  object 
 13  latitude        72331 non-null  float64
 14  longitude       72331 non-null  float64
 15  reply_count     72331 non-null  int64  
 16  quote_count     72331 non-null  int64  
 17  likes_count     72331 non-null 

In [51]:
# Confirm dataframe and fips values are corrected visually
tweet_df.head(10)

,tweet_id,date_created,full_text,tweet_type,hashtags,mentions,user_id,city,county,fips_county,...,fips_state,country,latitude,longitude,reply_count,quote_count,likes_count,retweet_counts,hyperlink,sentiment
0,1588320083335467009,2022-11-03 23:59:52,"""@UnnecRoughness A is Levis. \nB is KJ Jeffers...",Reply Tweet,NaN,['@UnnecRoughness'],531999121,NaN,NaN,NaN,...,05,United States,-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...,neutral
1,1588320019259469824,2022-11-03 23:59:36,"""Republican candidate's kids are almost KILLED...",Original Tweet,NaN,NaN,1586027897579802624,NaN,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...,neutral
2,1588320006840160256,2022-11-03 23:59:33,"""@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy...",Reply Tweet,NaN,"['@Jupiter62214807', '@cjstheman_611', '@BMC_M...",783106891274596352,NaN,NaN,NaN,...,54,United States,-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...,neutral
3,1588319891446272001,2022-11-03 23:59:06,"""@davidhogg111 It’s 2022, get new talking poin...",Reply Tweet,NaN,['@davidhogg111'],1511111650199412739,NaN,NaN,NaN,...,26,United States,-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...,neutral
4,1588319866385477632,2022-11-03 23:59:00,"""Watch: Gunman opens fire on Imran Khan https:...",Original Tweet,NaN,NaN,287297482,NaN,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...,neutral
5,1588319827651104769,2022-11-03 23:58:51,"""@Stormof69 @VoteYesOn114 I’m always amused th...",Reply Tweet,NaN,"['@Stormof69', '@VoteYesOn114']",412772316,Manchester,Hartford County,09003,...,09,United States,-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...,pro-gun
6,1588319810181529602,2022-11-03 23:58:46,"""@ColinCowherd @DrHoodieMaddog What about kids...",Reply Tweet,NaN,"['@ColinCowherd', '@DrHoodieMaddog']",147647057,Brooklyn,Kings County,36047,...,36,United States,-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...,neutral
7,1588319807375433728,2022-11-03 23:58:46,"""@Unnoble_Savage @brianleeellis @michaeljknowl...",Reply Tweet,NaN,"['@Unnoble_Savage', '@brianleeellis', '@michae...",1040679839747825664,Willits,Mendocino County,06045,...,06,United States,-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...,pro-gun
8,1588319706343124992,2022-11-03 23:58:22,"""@RobSchneider Let’s keep the 2nd Amendment an...",Reply Tweet,NaN,['@RobSchneider'],1586158533149499392,Monterey,Monterey County,06053,...,06,United States,-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...,neutral
9,1588319697808019456,2022-11-03 23:58:20,"""@davidhogg111 How many times a year are guns ...",Reply Tweet,NaN,['@davidhogg111'],1527469162838253568,NaN,NaN,NaN,...,NaN,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...,anti-gun


### Loading Dataframe to PostgreSQL/AWS-RDS Database

In [55]:
# Connect to PostrgreSQL database server on AWS-RDS

user = 'postgres'

password = getpass('Enter database password')

conn_string = f'postgresql://{user}:{password}\
@gun-sentiment-db.ckbz1nmb8sin.us-west-2.rds.amazonaws.com/election_gun_sentiment_tweets_db'
  
 
db = create_engine(conn_string)
conn = db.connect()


# Write tweet_df to database
tweet_df.to_sql('tweets', con=conn, if_exists='replace',
          index=False)
conn = psycopg2.connect(conn_string
                        )
conn.autocommit = True
cursor = conn.cursor()
  
sql1 = '''select * from tweets limit 20;'''
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)

# conn.commit()
conn.close()

Enter database password ··············


('1588320083335467009', '2022-11-03 23:59:52', '"@UnnecRoughness A is Levis. \nB is KJ Jefferson. \nThe same people who think Levis is an NFL Qb is the same people who think AR-15 is. \nThey will both be bust in 3-5 years. https://t.co/G9KGUy7hbm"', 'Reply Tweet', None, "['@UnnecRoughness']", '531999121', None, None, None, 'Arkansas', '05', 'United States', -92.50044, 34.75037, 0, 0, 6, 0, 'https://twitter.com/twitter/status/1588320083335467009', 'neutral')
('1588320019259469824', '2022-11-03 23:59:36', '"Republican candidate\'s kids are almost KILLED after gunman opens fire at his North Carolina home with bullet landing few feet from where they slept \n\n- after Democrat rival filmed campaign ad outside his property https://t.co/WY6kAqT9Ge"', 'Original Tweet', None, None, '1586027897579802624', None, None, None, None, None, 'United States', -98.5, 39.76, 0, 0, 0, 0, 'https://twitter.com/twitter/status/1588320019259469824', 'neutral')
('1588320006840160256', '2022-11-03 23:59:33', '"@J

In [56]:
# Write tweet_df to final sentiment analysis CSV
tweet_df.to_csv('../res/final_tweets_sentiment_analysis.csv', index=False)